# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup

In [2]:
import openai
import os

openai.api_key = os.environ.get('OPENAI_API_KEY')

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{'role': "user", "content": prompt}]
    response = openai.ChatCompletion.create(model=model,
                                            messages=messages, temperature=0)
    return response.choices[0].message["content"]

## Product review text

In [4]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [5]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.


限定输出指令 positive  or negative，这样输出结果唯一且确定，便于程序做后续处理

In [6]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

positive


## Identify types of emotions

In [6]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, grateful, impressed, content


## Identify anger

In [7]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


In [8]:
review_1 = "做工依旧精致，买来搭配颂拓Vertical，基本没有色差，理论上22mm通用。个人觉得比上次买的适配三星的表带还要好，拆卸表带不需要额外工具了，很方便，这点必须好评"
review_2 = "重量和他们发出的一样。产品不一样然后怨我购买他们家产品了。我要是知道这样我会买吗 这不是骗人吗。 他们发出的重量和我收到的一样。到现在没人给个说话。重量是一模一样的 他们现在就是推给快递出错了。就是不说他们出错了。服*了"
review_3 = "实物与描述差异很大，做工极差。还打着中国梦的旗号来招摇撞骗。机器底部一块黑色脏的，结构件严重不贴合，机壳就是一个廉价塑料。还说拿了些什么奖。虚假宣传，误导消费者。让他们拿出来都拿不出来。也不知道这些销量和其他评价的水分有多重。目前正在找淘宝维权中。。。"
reviews = [review_1, review_2, review_3]

In [13]:
for i in range(len(reviews)):
    prompt = f"""
    客户评论是好评还是差评。
    用“好评” 或 “差评”回答，不要带任何标点。

    Review text: '''{reviews[i]}'''
    """
    response = get_completion(prompt)
    print(i, response, "\n")

0 好评 

1 差评 

2 差评 



In [14]:
for i in range(len(reviews)):
    prompt = f"""
    请用5个关键词总结用户的评论，评论用三个单引号隔开。
    结果返回一个python list。

    Review text: '''{reviews[i]}'''
    """
    response = get_completion(prompt)
    print(i, response, "\n")

0 ['精致做工', '无色差', '22mm通用', '易拆卸', '好评'] 

1 ['重量', '产品', '不一样', '骗人', '快递出错'] 

2 ['实物描述差异', '做工极差', '虚假宣传', '误导消费者', '维权中'] 



## Extract product and company name from customer reviews
从文本中提取指定的信息，比如品牌，产品名称等参数，并要求结构化输出。

In [17]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


## Doing multiple tasks at once

In [18]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp with additional storage",
  "Brand": "Lumina"
}


## Inferring topics

In [20]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

## Infer 5 topics
有大量文章，可以用LLM提取topic,并建立索引

In [21]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

government survey, job satisfaction, NASA, Social Security Administration, employee concerns


In [22]:
response.split(sep=',')

['government survey',
 ' job satisfaction',
 ' NASA',
 ' Social Security Administration',
 ' employee concerns']

In [23]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [24]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


In [25]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


In [26]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as a json. keys are from the list of topics, and value is 0 or 1 for ture or false.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

{
  "nasa": 1,
  "local government": 0,
  "engineering": 0,
  "employee satisfaction": 1,
  "federal government": 1
}
